In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
def create_rotation_matrix(angle, n, i_in, j_in):
    R = []
    for i in range(n):
        z = []
        for j in range(n):
            if i == j:
                z.append(1)
            else:
                z.append(0)
        R.append(z)

    R[i_in][i_in] = tf.cos(angle)
    R[i_in][j_in] = -tf.sin(angle)
    R[j_in][i_in] = tf.sin(angle)
    R[j_in][j_in] = tf.cos(angle)
    return tf.reshape(tf.stack(R),[n,n])

def create_givens_matrix(angles, n, p):
    G = tf.constant(np.eye(n), dtype = tf.float32)
    for j in range(0, n-1):
        R = create_rotation_matrix(angles[j], n, 0, j+1)
        G = tf.matmul(G, R)
    
    return G[:,:p]

def create_unit_vector(angle):
    return tf.stack([tf.sin(angle), tf.cos(angle)])

def create_unit_vector_np(angle):
    return np.array([np.cos(angle), np.sin(angle)])

In [2]:
def ordered_transform(y):
    G = []
    if len(y.shape) == 2:
        for i in range(y.shape[0]):
            if i == 0:
                G.append(y[i,:])
            else:
                G.append(G[i-1] + tf.exp(y[i,:]))
    else:
        for i in range(y.shape[0]):
            if i == 0:
                G.append(y[i])
            else:
                G.append(G[i-1] + tf.exp(y[i]))
    return tf.stack(G)

In [3]:
tf.reset_default_graph()
initializer = tf.contrib.layers.xavier_initializer()

input = tf.placeholder(dtype = tf.float32, shape = [None,784])
targets = tf.placeholder(dtype = tf.float32, shape = [None, 10])

W1_unconstrained = tf.Variable(initializer([784,20]))
b1_unconstrained = tf.Variable(initializer([20]))
b1 = ordered_transform(b1_unconstrained)
W2_unconstrained = tf.Variable(initializer([20,20]))
b2_unconstrained = tf.Variable(initializer([20]))
b2 = ordered_transform(b2_unconstrained)
W3_unconstrained = tf.Variable(initializer([20,10]))
b3_unconstrained = tf.Variable(initializer([10]))
b3 = ordered_transform(b3_unconstrained)

h1 = tf.nn.relu(tf.matmul(input, W1_unconstrained) + b1)
h2 = tf.nn.relu(tf.matmul(h1, W2_unconstrained) + b2)
output = tf.matmul(h2, W3_unconstrained) + b3

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=targets))
train_step = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(loss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(output, 1), tf.argmax(targets, 1)), tf.float32))

In [4]:
tf.set_random_seed(100)
mnist = input_data.read_data_sets("../datasets/MNIST_data/", one_hot=True, seed=50)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(100):
    batch_xs, batch_ys = mnist.train.next_batch(512)
    _, l,a = sess.run([train_step,loss,accuracy], feed_dict = {input : batch_xs, targets: batch_ys})
    if i % 10 == 0:
        print("Iter %d, Loss %f, Accuracy %f%%" % (i, l, a*100))

Extracting ../datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
Iter 0, Loss 47.845379, Accuracy 11.914062%
Iter 10, Loss 5.444014, Accuracy 25.000000%
Iter 20, Loss 2.661141, Accuracy 43.750000%
Iter 30, Loss 1.629519, Accuracy 52.734375%
Iter 40, Loss 1.067997, Accuracy 69.140625%
Iter 50, Loss 0.824954, Accuracy 78.515625%
Iter 60, Loss 0.746834, Accuracy 77.734375%
Iter 70, Loss 0.587426, Accuracy 84.375000%
Iter 80, Loss 0.567643, Accuracy 80.664062%
Iter 90, Loss 0.507001, Accuracy 84.960938%


In [5]:
print("Test Accuracy %f" % sess.run(accuracy, feed_dict = {input: mnist.test.images, targets: mnist.test.labels}))

0.86860001